In [1]:
from google.colab import drive

# Step 1: Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch

def load_image(image_path):
    return Image.open(image_path).convert("RGB")

def generate_caption(image_path):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    image = load_image(image_path)
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        output = model.generate(**inputs)

    caption = processor.batch_decode(output, skip_special_tokens=True)[0]
    return caption

if __name__ == "__main__":
    #image_path = "/content/drive/My Drive/GenAIWorkshop/I3.jfif"  # Replace with your image path
   # image_path = "/content/drive/My Drive/GenAIWorkshop/CT/n8.jpg"  # Replace with your image path
    image_path = "/content/drive/My Drive/dogs_n_cats/cats_set/cat.4002.jpg"  # Replace with your image path
    caption = generate_caption(image_path)
    print("Generated Caption:", caption)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

Generated Caption: a man holding a cat in his hands


In [4]:
pip install pydicom


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.5 MB/s eta 0:00:00


In [6]:
pip install gdcm pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg numpy


In [ ]:
# Medical Image Caption


from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import torch
import pydicom
import numpy as np

def load_medical_image(image_path):
    dicom_image = pydicom.dcmread(image_path, force=True)  # Force read even if metadata is missing

    # If TransferSyntaxUID is missing, set a default
    if "TransferSyntaxUID" not in dicom_image.file_meta:
        print("Warning: Missing Transfer Syntax UID. Assuming Explicit VR Little Endian.")
        dicom_image.file_meta.TransferSyntaxUID = pydicom.uid.ExplicitVRLittleEndian

    # Ensure pixel data exists
    if not hasattr(dicom_image, "PixelData"):
        raise ValueError("DICOM file does not contain pixel data.")

    try:
        # Handle compressed images
        if dicom_image.file_meta.TransferSyntaxUID.is_compressed:
            dicom_image.decompress()

        image_array = dicom_image.pixel_array  # Extract pixel data
    except Exception as e:
        raise ValueError(f"Failed to read pixel data: {e}")

    # Normalize pixel values and convert to RGB image
    image = Image.fromarray((image_array / np.max(image_array) * 255).astype(np.uint8)).convert("RGB")
    return image

def generate_medical_caption(image_path):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

    image = load_medical_image(image_path)
    inputs = processor(images=image, return_tensors="pt")

    with torch.no_grad():
        output = model.generate(**inputs)

    caption = processor.batch_decode(output, skip_special_tokens=True)[0]
    return caption

if __name__ == "__main__":
    image_path = "/content/drive/My Drive/hum0102.dcm" # Replace with your DICOM image path
    try:
        caption = generate_medical_caption(image_path)
        print("Generated Caption:", caption)
    except Exception as e:
        print("Error:", str(e))
